# LCEL的好处

以下是 **langchain 文档的原文**摘取：

**LangChain 表达式语言（LCEL）是一种轻松地将链组合在一起的声明性方式。**

LCEL 从第一天起就被设计为支持将原型投入生产，无需更改代码，从最简单的 **「Prompt + Model」** 链到最复杂的链（我们已经看到人们在生产中成功运行了 100 个步骤的 LCEL 链）。 

强调一下您可能想要使用 LCEL 的一些原因：


**流支持**<br>
当您使用 LCEL 构建链时，您可以获得最佳的首次Token时间（直到第一个输出块出现之前经过的时间）。 对于某些连锁店来说，这意味着例如。 我们将令牌直接从 LLM 流式传输到流式输出解析器，然后您会以与 LLM 提供者输出原始令牌相同的速率返回已解析的增量输出块。

**异步支持**<br>
使用 LCEL 构建的任何链都可以使用同步 API（例如，在原型设计时在 Jupyter 笔记本中）和异步 API（例如，在 LangServe 服务器中）进行调用。 这使得能够在原型和生产中使用相同的代码，具有出色的性能，并且能够在同一服务器中处理许多并发请求。

**优化的并行执行**<br>
只要您的 LCEL 链具有可以并行执行的步骤（例如，如果您从多个检索器获取文档），我们就会在同步和异步接口中自动执行此操作，以尽可能减少延迟。

**重试和回退**<br>
为 LCEL 链的任何部分配置重试和回退。 这是让您的链条在规模上更加可靠的好方法。 我们目前正在努力添加对重试/回退的流支持，以便您可以获得更高的可靠性，而无需任何延迟成本。

**访问中间结果**<br>
对于更复杂的链，即使在生成最终输出之前访问中间步骤的结果通常也非常有用。 这可以用来让最终用户知道正在发生的事情，甚至只是为了调试您的链。 您可以流式传输中间结果，并且它在每个 LangServe 服务器上都可用。

**输入和输出模式**<br>
输入和输出模式为每个 LCEL 链提供从链结构推断出的 Pydantic 和 JSONSchema 模式。 这可用于验证输入和输出，并且是 LangServe 的组成部分。

# 基本模式

## Prompt + Model + OutputParse

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo-1106")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

'Why did the ice cream go to therapy? Because it was feeling a little sundae-pressed!'

## Prompt

Prompt 是一个 BasePromptTemplate，这意味着它接受模板变量的字典并生成 PromptValue。 
PromptValue 是完整提示的包装器，可以传递给 LLM（将字符串作为输入）或 ChatModel（将一系列消息作为输入）。 
它可以使用任一语言模型类型，因为它定义了用于生成 BaseMessage 和生成字符串的逻辑。

In [5]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

## Model

然后将 PromptValue 传递给模型。 在本例中，我们的模型是 ChatModel，这意味着它将输出 BaseMessage。

In [6]:
message = model.invoke(prompt_value)
message

AIMessage(content='Why did the ice cream go to therapy? Because it had too many sprinkles of anxiety!')

In [7]:
# 如果不是对话模型，而是LLM模型，就会输出字符串

from langchain_openai.llms import OpenAI
llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

' \n\nAI: Why did the ice cream go to therapy? Because it had a rocky road!'

## OutputParser

最后，我们将模型输出传递给 output_parser，它是一个 BaseOutputParser，这意味着它接受字符串或 BaseMessage 作为输入。
StrOutputParser 特别简单地将任何输入转换为字符串。

In [8]:
output_parser.invoke(message)

'Why did the ice cream go to therapy? Because it had too many sprinkles of anxiety!'